In [3]:
import numpy as np
import os
import fileinput
import sys
import pandas as pd
import matplotlib.pyplot as plt

from utilities.foam_automation.caserunner import run_until_convergence, get_endtime
from utilities.foam_automation.computeFoamLossFunc import computeFoamFieldLossFunc
from utilities.foam_automation.set_foam_coeff import set_foam_coef

from utilities.json_io import write_suggestion, write_json, load_coeff_default, load_coeff_bounds, load_history_loss_log, load_suggestion

from utilities.control import initialize
from update_history import register_score
from suggest_coeffs import suggest
import random

# Bayesian optimization

In [3]:
foamdir = '/home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_template'
tunerdir = os.path.join(foamdir,'tuner')
ref_df = pd.read_csv(os.path.join(tunerdir,'refdata.csv'))

ref_df.describe()
foamLoss = computeFoamFieldLossFunc(foamdir=foamdir,
                                    ref_df=ref_df,
                                    interp_method='nearest',read_cell_centres=False)
foamLoss.foam_general_loss_function(foamtime = '0',
                                    coef_default_dict={'a1':0.31,'betaStar':0.09},
                                    coef_dict = {'a1':0.32, 'betaStar':0.15},
                                    error_calc_fields=[],
                                    error_calc_intparams=['Cl','Cd'])

1.0209
1.103368
0.01928
0.009221445
error_term 0.602488988414949
coef_term 0.34946236559139787
score 0.7077626019820773


0.7077626019820773

In [2]:
# Run simulation loop
foamdir = '/home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_50iter'
savedir = '/home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_50iter_opt'

tunerdir = os.path.join(foamdir,'tuner')
iterations=50
n_points_sampled = 4
utility = 'poi'
xi = 0.1
ref_df = pd.read_csv(os.path.join(tunerdir,'refdata.csv'))


initialize(directory = tunerdir,
           coeff_bounds = None,
           coeff_default = None,
           restart = True)

foamLoss = computeFoamFieldLossFunc(foamdir=foamdir,
                                    ref_df=ref_df,
                                    interp_method='nearest',
                                    read_cell_centres=False)
np.random.seed(7)
a1_rand = np.random.uniform(.248, .372, size=(n_points_sampled))
betaStar_rand = np.random.uniform(.072, .108, size=(n_points_sampled))
score_best = 1E6 

for i in range(iterations):
    if i == 0:
        search_point=load_coeff_default(directory=tunerdir)
        write_suggestion(directory=tunerdir, suggestion=search_point)
    elif i < n_points_sampled:
        search_point = {"a1": a1_rand[i], "betaStar": betaStar_rand[i]}
        write_suggestion(directory=tunerdir, suggestion=search_point)
    else:
        suggest(directory=tunerdir, random_state=7,utility_kind = utility, xi=xi)
    
    search_point = load_suggestion(directory=tunerdir)
    print(search_point)
    set_foam_coef(foamdir,search_point)
    last_time = run_until_convergence(case=foamdir, n_proc = 8)
    score = -foamLoss.foam_gedcp(foamtime=last_time,
                                                   coef_default_dict = load_coeff_default(directory=tunerdir),
                                                   coef_dict=search_point,
                                                   error_calc_fields=[],
                                                   error_calc_intparams=['Cl'],
                                                   error_type = 'mape'
                                                  )
                                                   
    register_score(score=score, directory=tunerdir)
    if abs(score) < abs(score_best):   
        score_best = score
        os.system(f'rm -r {savedir}')
        os.system(f'cp -r {foamdir} {savedir}')

    

INFO:general:Starting: decomposePar -force -case /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_50iter in /home/ryley/ML/turbo-rans
INFO:general:Started with PID 28615


{'a1': 0.31, 'betaStar': 0.09}


INFO:general:Finished
INFO:general:Starting: mpirun -n 8 simpleFoam -parallel -case /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_50iter in /home/ryley/ML/turbo-rans
INFO:general:Started with PID 28620
INFO:general:Finished
INFO:general:Starting: reconstructPar -latestTime -case /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_50iter in /home/ryley/ML/turbo-rans
INFO:general:Started with PID 28655
INFO:general:Finished
INFO:general:Starting: decomposePar -force -case /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_50iter in /home/ryley/ML/turbo-rans
INFO:general:Started with PID 28666


{'a1': 0.3447099302377742, 'betaStar': 0.09138585133477561}


INFO:general:Finished
INFO:general:Starting: mpirun -n 8 simpleFoam -parallel -case /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_50iter in /home/ryley/ML/turbo-rans
INFO:general:Started with PID 28671
INFO:general:Finished
INFO:general:Starting: reconstructPar -latestTime -case /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_50iter in /home/ryley/ML/turbo-rans
INFO:general:Started with PID 28706
INFO:general:Finished
INFO:general:Starting: decomposePar -force -case /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_50iter in /home/ryley/ML/turbo-rans
INFO:general:Started with PID 28713


...

INFO:general:Finished
INFO:general:Starting: mpirun -n 8 simpleFoam -parallel -case /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_50iter in /home/ryley/ML/turbo-rans
INFO:general:Started with PID 28718
INFO:general:Finished
INFO:general:Starting: reconstructPar -latestTime -case /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_50iter in /home/ryley/ML/turbo-rans
INFO:general:Started with PID 28753
INFO:general:Finished
INFO:general:Starting: decomposePar -force -case /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_50iter in /home/ryley/ML/turbo-rans
INFO:general:Started with PID 28760


{'a1': 0.3377096820510367, 'betaStar': 0.07459384080095141}


INFO:general:Finished
INFO:general:Starting: mpirun -n 8 simpleFoam -parallel -case /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_50iter in /home/ryley/ML/turbo-rans
INFO:general:Started with PID 28765
INFO:general:Finished
INFO:general:Starting: reconstructPar -latestTime -case /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_airfoil/airfoil_50iter in /home/ryley/ML/turbo-rans
INFO:general:Started with PID 28801
INFO:general:Finished


 Interrupted by the Keyboard
Killing PID 28765
{'a1': 0.3023664693596941, 'betaStar': 0.09008447261706655}



KeyboardInterrupt

